In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import joblib

In [ ]:
# Used to generate the d_min, time_to_closest and relative velocity
import numpy as np
import pandas as pd

def append_collision_features(df, threshold=1000):
    new_rows = []

    # group by epoch
    for epoch, group in df.groupby("epoch"):
        objs = group.to_dict("records")

        for i, obj1 in enumerate(objs):
            r1 = np.array([obj1["x"], obj1["y"], obj1["z"]])
            v1 = np.array([obj1["vx"], obj1["vy"], obj1["vz"]])

            closest_dist = np.inf
            closest_t = None
            closest_vel = None

            for j, obj2 in enumerate(objs):
                if i == j:  # skip self
                    continue

                r2 = np.array([obj2["x"], obj2["y"], obj2["z"]])
                v2 = np.array([obj2["vx"], obj2["vy"], obj2["vz"]])

                r_rel = r1 - r2
                v_rel = v1 - v2

                # time of closest approach
                if np.linalg.norm(v_rel) > 0:
                    t_star = -np.dot(r_rel, v_rel) / np.linalg.norm(v_rel) ** 2
                    if t_star < 0:  
                        t_star = 0
                else:
                    t_star = 0

                r_closest = r_rel + v_rel * t_star
                d_min = np.linalg.norm(r_closest)

                if d_min < closest_dist:
                    closest_dist = d_min
                    closest_t = t_star
                    closest_vel = np.linalg.norm(v_rel)

            # create updated row with appended features
            updated_row = obj1.copy()
            updated_row.update({
                "d_min": closest_dist,
                "time_to_closest": closest_t,
                "rel_velocity": closest_vel,
                "collision_risk": 1 if closest_dist < threshold else 0
            })
            new_rows.append(updated_row)

    return pd.DataFrame(new_rows)

In [5]:
new_space_data = pd.read_csv("Space_Data.csv")

In [6]:
def generate_ml_features(df):
    """
    Generate leakage-free, predictive features for collision risk modeling.
    Each row is an object at an epoch, with aggregated features relative to all other objects.
    """
    new_rows = []

    for epoch, group in df.groupby("epoch"):
        objs = group.to_dict("records")

        for i, obj1 in enumerate(objs):
            # skip object if it has zero values in any key field
            if any(obj1[f] == 0 for f in ["x","y","z","vx","vy","vz"]):
                continue

            r1 = np.array([obj1["x"], obj1["y"], obj1["z"]], dtype=float)
            v1 = np.array([obj1["vx"], obj1["vy"], obj1["vz"]], dtype=float)

            features_list = []

            for j, obj2 in enumerate(objs):
                if i == j:
                    continue
                # skip if object2 has zero values
                if any(obj2[f] == 0 for f in ["x","y","z","vx","vy","vz"]):
                    continue

                r2 = np.array([obj2["x"], obj2["y"], obj2["z"]], dtype=float)
                v2 = np.array([obj2["vx"], obj2["vy"], obj2["vz"]], dtype=float)

                delta_r = r1 - r2
                delta_v = v1 - v2
                rel_vel = np.linalg.norm(delta_v)

                # angle between velocity vectors
                dot_prod = np.dot(v1, v2)
                norm_prod = np.linalg.norm(v1) * np.linalg.norm(v2)
                angle_vel = np.arccos(np.clip(dot_prod / norm_prod, -1, 1)) if norm_prod != 0 else 0

                features_list.append({
                    "rel_velocity": rel_vel,
                    "delta_x": delta_r[0],
                    "delta_y": delta_r[1],
                    "delta_z": delta_r[2],
                    "angle_vel": angle_vel
                })

            if not features_list:
                continue  # skip if no valid comparisons

            # aggregate features across all other objects
            row_features = obj1.copy()
            row_features.update({
                "rel_velocity_max": max(f["rel_velocity"] for f in features_list),
                "rel_velocity_mean": np.mean([f["rel_velocity"] for f in features_list]),
                "delta_x_min": min(f["delta_x"] for f in features_list),
                "delta_y_min": min(f["delta_y"] for f in features_list),
                "delta_z_min": min(f["delta_z"] for f in features_list),
                "angle_vel_max": max(f["angle_vel"] for f in features_list),
                "angle_vel_mean": np.mean([f["angle_vel"] for f in features_list])
            })

            # keep the label for supervised training
            row_features["collision_risk"] = obj1.get("collision_risk", 0)

            new_rows.append(row_features)

    return pd.DataFrame(new_rows)

In [7]:
space_data = generate_ml_features(new_space_data)

In [15]:
space_data.drop(['Unnamed: 0','d_min','time_to_closest'],axis=1,inplace=True)

In [16]:
space_data.columns

Index(['classificationMarking_x', 'object_id', 'pointId', 'x', 'y', 'z', 'vx',
       'vy', 'vz', 'source_x', 'pointStartTime', 'pointEndTime',
       'referenceFrame', 'source_y', 'epoch', 'rel_velocity', 'collision_risk',
       'rel_velocity_max', 'rel_velocity_mean', 'delta_x_min', 'delta_y_min',
       'delta_z_min', 'angle_vel_max', 'angle_vel_mean'],
      dtype='object')

In [51]:
space_data.to_csv("Space_Data_with_collision_features.csv")

In [4]:
space_data = pd.read_csv("Space_Data_with_collision_features.csv")

In [5]:
space_data.head()

,Unnamed: 0,classificationMarking_x,object_id,pointId,x,y,z,vx,vy,vz,...,epoch,rel_velocity,collision_risk,rel_velocity_max,rel_velocity_mean,delta_x_min,delta_y_min,delta_z_min,angle_vel_max,angle_vel_mean
0,0,U,57f03102-f98b-43ad-93cb-3482c2ffd27f,eb1f9421-fe2f-4738-ada4-1c4771860a43,1300.680962,10966.10884,-5347.317939,-1.064615,2.438418,4.753137,...,2025-08-10 00:00:00+00:00,0.0,1,0.0,0.0,0.000060,0.00002,0.000056,0.0,0.0
1,1,U,976b0dab-713d-4696-947e-585686ba34da,5cec3e2e-96cf-4fd9-b7ab-7e4e7fc81c85,1300.680902,10966.10882,-5347.317995,-1.064615,2.438418,4.753137,...,2025-08-10 00:00:00+00:00,0.0,1,0.0,0.0,-0.000060,-0.00002,-0.000056,0.0,0.0
2,2,U,57f03102-f98b-43ad-93cb-3482c2ffd27f,a7099044-7e3a-402c-a464-bf0e7648b8a6,1173.489694,11243.06909,-4768.922726,-1.055470,2.176382,4.884289,...,2025-08-10 00:02:00+00:00,0.0,1,0.0,0.0,0.000061,0.00002,0.000058,0.0,0.0
3,3,U,976b0dab-713d-4696-947e-585686ba34da,77972af7-c988-4778-9e1f-96afcb622469,1173.489633,11243.06907,-4768.922784,-1.055470,2.176382,4.884289,...,2025-08-10 00:02:00+00:00,0.0,1,0.0,0.0,-0.000061,-0.00002,-0.000058,0.0,0.0
4,4,U,57f03102-f98b-43ad-93cb-3482c2ffd27f,6d12cdf0-0166-490a-8cf2-516cbebfcf50,1047.309966,11488.16798,-4175.695143,-1.047779,1.907534,5.000278,...,2025-08-10 00:04:00+00:00,0.0,1,0.0,0.0,0.000062,0.00001,0.000059,0.0,0.0


In [7]:
space_data.drop(['Unnamed: 0','classificationMarking_x','pointId'],axis=1,inplace=True)

In [8]:
train_df = space_data[space_data['epoch'] < '2025-08-24']
test_df  = space_data[space_data['epoch'] >= '2025-08-24']

X_train = train_df[["rel_velocity_max","rel_velocity_mean",
                    "delta_x_min","delta_y_min","delta_z_min",
                    "angle_vel_max","angle_vel_mean"]]
y_train = train_df["collision_risk"]

X_test = test_df[["rel_velocity_max","rel_velocity_mean",
                  "delta_x_min","delta_y_min","delta_z_min",
                  "angle_vel_max","angle_vel_mean"]]
y_test = test_df["collision_risk"]

In [21]:
# Random Forest
rf = RandomForestClassifier(n_estimators=200, class_weight="balanced")
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest:\n", classification_report(y_test, y_pred_rf))

Random Forest:
               precision    recall  f1-score   support

           0       0.69      0.70      0.70     18120
           1       0.94      0.94      0.94     90312

    accuracy                           0.90    108432
   macro avg       0.82      0.82      0.82    108432
weighted avg       0.90      0.90      0.90    108432



In [29]:
accuracy = accuracy_score(y_test,y_pred_rf)
print("Accuracy: " , accuracy)
precision = precision_score(y_test,y_pred_rf)
print("Precision: " ,precision)
recall = recall_score(y_test,y_pred_rf)
print("Recall " ,recall)

Accuracy:  0.8980005902316659
Precision:  0.9395953051851523
Recall  0.9378266454070334


In [30]:
f1_score(y_test, y_pred_rf)

0.9387101421969033

In [23]:
# XGBoost
xgb = xgb.XGBClassifier(n_estimators=200, scale_pos_weight=1)  # tune scale_pos_weight if imbalance
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print("XGBoost:\n", classification_report(y_test, y_pred_xgb))

C:\Users\kiimi\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:44:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\kiimi\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


XGBoost:
               precision    recall  f1-score   support

           0       0.81      0.64      0.71     18120
           1       0.93      0.97      0.95     90312

    accuracy                           0.91    108432
   macro avg       0.87      0.80      0.83    108432
weighted avg       0.91      0.91      0.91    108432



In [31]:
accuracy = accuracy_score(y_test,y_pred_xgb)
print("Accuracy: " , accuracy)
precision = precision_score(y_test,y_pred_xgb)
print("Precision: " ,precision)
recall = recall_score(y_test,y_pred_xgb)
print("Recall " ,recall)

Accuracy:  0.9140382912793271
Precision:  0.93035909752702
Recall  0.9693506953671716


In [32]:
f1_score(y_test, y_pred_xgb)

0.9494547446165859

In [33]:
rf2 = RandomForestClassifier(n_estimators=100, max_depth = 10, random_state= 60, min_samples_split=6,min_samples_leaf=8)
rf2.fit(X_train, y_train)
y_pred_rf2 = rf2.predict(X_test)
print("Random Forest:\n", classification_report(y_test, y_pred_rf2))

Random Forest:
               precision    recall  f1-score   support

           0       0.69      0.70      0.70     18120
           1       0.94      0.94      0.94     90312

    accuracy                           0.90    108432
   macro avg       0.82      0.82      0.82    108432
weighted avg       0.90      0.90      0.90    108432



In [34]:
y_pred_rf2 = rf2.predict(X_test)
print("Random Forest:\n", classification_report(y_test, y_pred_rf2))

Random Forest:
               precision    recall  f1-score   support

           0       0.88      0.54      0.67     18120
           1       0.92      0.98      0.95     90312

    accuracy                           0.91    108432
   macro avg       0.90      0.76      0.81    108432
weighted avg       0.91      0.91      0.90    108432



In [35]:
accuracy2 = accuracy_score(y_test,y_pred_rf2)
print("Accuracy: " , accuracy2)
precision2 = precision_score(y_test,y_pred_rf2)
print("Precision: " ,precision2)
recall2 = recall_score(y_test,y_pred_rf2)
print("Recall " ,recall2)

Accuracy:  0.9112070237568246
Precision:  0.915152201206084
Recall  0.9846864204092479


In [36]:
f1_score(y_test, y_pred_rf2)

0.9486468322968115

In [37]:
xgb2 = xgb.XGBClassifier(n_estimators=100, random_state= 60, learning_rate=0.1, max_depth=5, eta=0.2, subsample=0.5, reg_lambda=0.3)
xgb2.fit(X_train, y_train)
y_pred_xgb2 = xgb2.predict(X_test)
print("XGBoost:\n", classification_report(y_test, y_pred_xgb2))

C:\Users\kiimi\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[07:57:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\kiimi\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


XGBoost:
               precision    recall  f1-score   support

           0       0.87      0.58      0.70     18120
           1       0.92      0.98      0.95     90312

    accuracy                           0.92    108432
   macro avg       0.90      0.78      0.82    108432
weighted avg       0.91      0.92      0.91    108432



In [38]:
accuracy3 = accuracy_score(y_test,y_pred_xgb2)
print("Accuracy: " , accuracy3)
precision3 = precision_score(y_test,y_pred_xgb2)
print("Precision: " ,precision3)
recall3 = recall_score(y_test,y_pred_xgb2)
print("Recall " ,recall3)

Accuracy:  0.915762874428213
Precision:  0.921625046744505
Recall  0.9824054389228453


In [39]:
f1_score(y_test, y_pred_xgb2)

0.9510451280951872

In [42]:
filename = 'random_forest_model2.joblib'
joblib.dump(rf, filename)

['random_forest_model2.joblib']

In [9]:
xgb3 = xgb.XGBClassifier(n_estimators=200, random_state= 60, learning_rate=0.1, max_depth=5, eta=0.5)
xgb3.fit(X_train, y_train)
y_pred_xgb3 = xgb3.predict(X_test)
print("XGBoost:\n", classification_report(y_test, y_pred_xgb3))

C:\Users\kiimi\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:55:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\kiimi\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


XGBoost:
               precision    recall  f1-score   support

           0       0.86      0.60      0.71     18120
           1       0.92      0.98      0.95     90312

    accuracy                           0.92    108432
   macro avg       0.89      0.79      0.83    108432
weighted avg       0.91      0.92      0.91    108432



In [10]:
accuracy4 = accuracy_score(y_test,y_pred_xgb3)
print("Accuracy: " , accuracy4)
precision4 = precision_score(y_test,y_pred_xgb3)
print("Precision: " ,precision4)
recall4 = recall_score(y_test,y_pred_xgb3)
print("Recall " ,recall4)

Accuracy:  0.9178747971078648
Precision:  0.9249287496476631
Recall  0.9810213482150766


In [47]:
f1_score(y_test, y_pred_xgb3)

0.9521496391744267

In [11]:
filename = 'xgb_model2.joblib'
joblib.dump(xgb3, filename)

['xgb_model2.joblib']

In [49]:
rf3 = RandomForestClassifier(n_estimators=200, max_depth = 5, random_state= 60, min_samples_split=5,min_samples_leaf=5)
rf3.fit(X_train, y_train)
y_pred_rf3 = rf3.predict(X_test)
print("Random Forest:\n", classification_report(y_test, y_pred_rf3))

Random Forest:
               precision    recall  f1-score   support

           0       0.92      0.42      0.58     18120
           1       0.90      0.99      0.94     90312

    accuracy                           0.90    108432
   macro avg       0.91      0.71      0.76    108432
weighted avg       0.90      0.90      0.88    108432



In [50]:
accuracy5 = accuracy_score(y_test,y_pred_rf3)
print("Accuracy: " , accuracy5)
precision5 = precision_score(y_test,y_pred_rf3)
print("Precision: " ,precision5)
recall5 = recall_score(y_test,y_pred_rf3)
print("Recall " ,recall5)

Accuracy:  0.89760402833112
Precision:  0.8958035598285046
Recall  0.9925037647267251


In [ ]:
f1_score(y_test, y_pred_rf3)

In [2]:
rf_model = joblib.load('random_forest_model2.joblib')

In [20]:
rf_model_predictions = rf_model.predict(X_test)

In [21]:
pd.DataFrame({'Row Number':X_test.index,'predictions':rf_model_predictions}).to_csv('RandomForestPredictions.csv')

In [16]:
X_test

Int64Index([413623, 413624, 413625, 413626, 413627, 413628, 413629, 413630,
            413631, 413632,
            ...
            522045, 522046, 522047, 522048, 522049, 522050, 522051, 522052,
            522053, 522054],
           dtype='int64', length=108432)

In [12]:
y_pred_xgb3

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [19]:
pd.DataFrame({'Row Number':X_test.index,'predictions':y_pred_xgb3}).to_csv('XGBoostPredictions.csv')